In [56]:
import cv2
import numpy as np

# 1. 3D World Coordinates (meters)
# (X, Y, Z) - We'll use X as long side, Y as wide side, Z as height
# Dimensions: 18m long, 9m wide, Net posts are 2.55m high
world_pts = np.array([
    [0, 0, 0],      # 1: Bottom Left Corner (Floor)
    [9, 0, 0],     # 2: Bottom Right Corner (Floor)
    [9, 9, 0],     # 3: Top Right Corner (Floor)
    [0, 9, 0],      # 4: Top Left Corner (Floor)
    [0, 0, 2.24],   # 5: Left Net Post Top (Z=2.55m)
    [9, 0, 2.24]    # 6: Right Net Post Top (Z=2.55m)
], dtype=np.float32)

# 2. Manual 2D Pixel Coordinates (Replace with your manual selections)
image_pts = np.array([
    [1159,1016],     # Corner 1 pixel
    [2645,1015],    # Corner 2 pixel
    [2368,594],    # Corner 3 pixel
    [1422,592],     # Corner 4 pixel
    [1094,643],     # Net Post 5 pixel
    [2713,644]     # Net Post 6 pixel
], dtype=np.float32)

In [57]:
# Assume a standard camera center (half of 1080p frame)
# and a guess for focal length if unknown. 
# solvePnP will refine the rotation and translation.
height, width = 2160,3840
camera_matrix = np.array([
    [width, 0, width / 2],
    [0, width, height / 2],
    [0, 0, 1]
], dtype=np.float32)

# Solve for rotation (rvec) and translation (tvec)
success, rvec, tvec = cv2.solvePnP(world_pts, image_pts, camera_matrix, None)

if success:
    print("Camera calibration successful!")
    print(f"Estimated Camera Distance: {np.linalg.norm(tvec):.2f} meters")

Camera calibration successful!
Estimated Camera Distance: 23.61 meters


In [54]:
def get_3d_height_from_size(ball_row, rvec, tvec, camera_matrix):
    """
    ball_row: detection row with 'x', 'y', and 'width'
    """

    # 1. Use ball width to get distance from camera
    # Focal length is the [0,0] element of our camera matrix
    focal_length = camera_matrix[0, 0]
    distance = (0.21 * focal_length) / 40
    
    # 2. Project the 2D pixel to a 3D vector from the camera
    pixel_point = np.array([2353, 691, 1.0]).reshape(3, 1)
    # Inverse of the camera matrix to get relative 3D direction
    direction_3d = np.dot(np.linalg.inv(camera_matrix), pixel_point)
    direction_3d /= np.linalg.norm(direction_3d) # Normalize
    
    # 3. Scale direction by the calculated distance
    ball_pos_rel_cam = direction_3d * distance
    
    # 4. Convert relative camera position to real World Coordinates (X, Y, Z)
    R, _ = cv2.Rodrigues(rvec)
    world_pos = np.dot(np.linalg.inv(R), (ball_pos_rel_cam - tvec))
    
    # The Z value is the height above the floor!
    return world_pos[2][0]

# Test it on your data
# df = pd.read_csv("output/table_data/game1_set1_cleaned.csv")
# df['height_m'] = df.apply(lambda row: get_3d_height_from_size(row, rvec, tvec, camera_matrix), axis=1)

In [ ]:
ball_row['width'] = 47
ball_pixel = [1832,412]

In [55]:
ball_row = [2353,691]
df = get_3d_height_from_size(ball_row, rvec, tvec, camera_matrix)
df

2.746297659749708

In [38]:
df

-0.1289187884961821

In [11]:
df

19.238668188560048